In [ ]:
# Демонстрация и проверка класса DataPreprocessor
# Загружаем тестовый датасет Titanic
# Смотрим на него изначально
# Применяем fit_transform()
# Сравниваем результат
# После также показаны разные дополнительное сведения

In [21]:
import pandas as pd
import seaborn as sns

from data_preprocessor import DataPreprocessor

In [22]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [32]:
print("shape:", df.shape)
df.info()

# доля пропуском по столбцам (от 0 до 1)
missing = df.isna().mean().sort_values(ascending=False)
missing[missing > 0]

shape: (891, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


,0
deck,0.772166
age,0.198653
embarked,0.002245
embark_town,0.002245


In [ ]:
# Ожидаем, что:
# Колонки с долей пропусков больше указанного threshold уйдут
# Остальные пропуски заполнятся
# Категории станут one-hot encoded
# Числовые значения нормализируются

In [33]:
dp = DataPreprocessor(df)
df_processed = dp.fit_transform(threshold=0.5, num_strategy="median", method="minmax")

df_processed.head()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_female,sex_male,...,class_Second,class_Third,who_child,who_man,who_woman,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alive_no,alive_yes
0,0.0,1.0,0.271174,0.125,0.0,0.014151,True,False,False,True,...,False,True,False,True,False,False,False,True,True,False
1,1.0,0.0,0.472229,0.125,0.0,0.139136,False,False,True,False,...,False,False,False,False,True,True,False,False,False,True
2,1.0,1.0,0.321438,0.000,0.0,0.015469,False,True,True,False,...,False,True,False,False,True,False,False,True,False,True
3,1.0,0.0,0.434531,0.125,0.0,0.103644,False,False,True,False,...,False,False,False,False,True,False,False,True,False,True
4,0.0,1.0,0.434531,0.000,0.0,0.015713,True,True,False,True,...,False,True,False,True,False,False,False,True,True,False


In [42]:
print("Shape before:", df.shape)
print("Shape after:", df_processed.shape)

print("Removed columns:", dp.removed_cols_)
print("NaNs before:", df.isna().sum().sum())
print("NaNs after :", df_processed.isna().sum().sum())

print("Количество колонок после one-hot:", len(dp.onehot_cols_))

Shape before: (891, 15)
Shape after: (891, 24)
Removed columns: ['deck']
NaNs before: 869
NaNs after : 0
Количество колонок после one-hot: 24


In [38]:
# Проверка нормализации

num_cols = df_processed.select_dtypes(include="number").columns
df_processed[num_cols].agg(["min", "max"]).head(15)

,survived,pclass,age,sibsp,parch,fare
min,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,1.0,1.0


In [39]:
# Проверка std-нормализации

dp2 = DataPreprocessor(df)
df_std = dp2.fit_transform(threshold=0.5, num_strategy="mean", method="std")

df_std.select_dtypes(include="number").agg(["mean", "std"]).head(15)

,survived,pclass,age,sibsp,parch,fare
mean,1.395567e-17,-4.386066e-17,2.153160e-16,6.379733e-17,3.588600e-17,-7.974666e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [41]:
# Сохраненные параметры
dp.filled_values_num_
dp.filled_values_cat_
dp.normalized_params_

{'method': 'minmax',
 'min': survived    0.00
 pclass      1.00
 age         0.42
 sibsp       0.00
 parch       0.00
 fare        0.00
 dtype: float64,
 'max': survived      1.0000
 pclass        3.0000
 age          80.0000
 sibsp         8.0000
 parch         6.0000
 fare        512.3292
 dtype: float64}